# Feature Engineering for Kalman Filter

Prepare data for hurricane track prediction using Kalman filter state-space model.


In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None


In [6]:
def load_ibtracs(path="ibtracs.ALL.list.v04r01.csv"):
    df = pd.read_csv(path, skiprows=[1], low_memory=False)
    df.columns = [c.lower() for c in df.columns]
    df = df.replace(' ', np.nan)
    
    df['storm_dir'] = pd.to_numeric(df['storm_dir'], errors='coerce').astype('Int64')
    df['storm_speed'] = pd.to_numeric(df['storm_speed'], errors='coerce').astype('Int64')
    df['usa_wind'] = pd.to_numeric(df['usa_wind'], errors='coerce').astype('Int64')
    df['usa_lat'] = pd.to_numeric(df['usa_lat'], errors='coerce').astype('Float64')
    df['usa_lon'] = pd.to_numeric(df['usa_lon'], errors='coerce').astype('Float64')
    df['iso_time'] = pd.to_datetime(df['iso_time'])
    
    return df

df = load_ibtracs("ibtracs.ALL.list.v04r01.csv")


In [7]:
# Filter to storms with sufficient observations for Kalman filter
obs_per_storm = df.groupby('sid').size()
valid_storms = obs_per_storm[obs_per_storm >= 2].index
df = df[df['sid'].isin(valid_storms)].copy()
df = df.sort_values(['sid', 'iso_time']).reset_index(drop=True)


In [8]:
def compute_velocity_from_positions(storm_data):
    """Compute velocity from position differences using haversine distance"""
    storm_data = storm_data.sort_values('iso_time').copy()
    
    dt = storm_data['iso_time'].diff().dt.total_seconds() / 3600  # hours
    dlat = storm_data['lat'].diff()
    dlon = storm_data['lon'].diff()
    
    # Handle longitude wrapping at ±180
    dlon = dlon.copy()
    dlon[dlon > 180] -= 360
    dlon[dlon < -180] += 360
    
    # Convert degrees to km (approximate)
    lat_km = dlat * 111.0
    lon_km = dlon * 111.0 * np.cos(np.radians(storm_data['lat']))
    
    # Compute speed (km/h) and direction (degrees)
    speed_kmh = np.sqrt(lat_km**2 + lon_km**2) / dt.replace(0, np.nan)
    direction = np.degrees(np.arctan2(lon_km, lat_km)) % 360
    
    # Convert speed to knots (1 knot = 1.852 km/h)
    speed_knots = speed_kmh / 1.852
    
    return speed_knots, direction

# Fill missing velocity for storms that need it
for sid in df['sid'].unique():
    storm = df[df['sid'] == sid]
    missing_mask = storm['storm_speed'].isna() | storm['storm_dir'].isna()
    
    if missing_mask.any():
        speed, direction = compute_velocity_from_positions(storm)
        df.loc[storm.index[missing_mask], 'storm_speed'] = speed[missing_mask].values
        df.loc[storm.index[missing_mask], 'storm_dir'] = direction[missing_mask].values


In [9]:
# Convert velocity to Cartesian components (v_lat, v_lon) in degrees per 6 hours
# This is more suitable for Kalman filter state vector
df['v_lat'] = df['storm_speed'] * 1.852 * 6 / 111.0 * np.cos(np.radians(df['storm_dir']))
df['v_lon'] = df['storm_speed'] * 1.852 * 6 / (111.0 * np.cos(np.radians(df['lat']))) * np.sin(np.radians(df['storm_dir']))


In [10]:
# Temporal features
df['storm_age_hours'] = df.groupby('sid')['iso_time'].transform(lambda x: (x - x.min()).dt.total_seconds() / 3600)
df['day_of_year'] = df['iso_time'].dt.dayofyear
df['month'] = df['iso_time'].dt.month


In [11]:
# Compute acceleration (change in velocity) for enhanced state representation
def compute_acceleration(storm_data):
    storm_data = storm_data.sort_values('iso_time').copy()
    dt = storm_data['iso_time'].diff().dt.total_seconds() / 3600
    
    dv_lat = storm_data['v_lat'].diff() / dt.replace(0, np.nan)
    dv_lon = storm_data['v_lon'].diff() / dt.replace(0, np.nan)
    
    return dv_lat, dv_lon

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    dv_lat, dv_lon = compute_acceleration(df.loc[storm_idx])
    df.loc[storm_idx, 'a_lat'] = dv_lat.values
    df.loc[storm_idx, 'a_lon'] = dv_lon.values


In [15]:
# Track curvature: measures how sharply storm is turning (indicates when linear assumption breaks)
def compute_track_curvature(storm_data):
    storm_data = storm_data.sort_values('iso_time').copy()
    
    ddir = storm_data['storm_dir'].diff()
    ddir = ddir.copy()
    ddir[ddir > 180] -= 360
    ddir[ddir < -180] += 360
    ddir = ddir.abs()
    
    speed = storm_data['storm_speed']
    curvature = ddir / (speed.replace(0, np.nan) + 1e-6)
    
    return curvature

for sid in df['sid'].unique():
    storm_idx = df[df['sid'] == sid].index
    df.loc[storm_idx, 'track_curvature'] = compute_track_curvature(df.loc[storm_idx]).values

# Latitude regime: affects storm motion characteristics
df['latitude_regime'] = pd.cut(df['lat'].abs(), bins=[0, 18, 28, 90], labels=[0, 1, 2], right=False).astype(float)

# Hemisphere indicator
df['hemisphere'] = (df['lat'] > 0).astype(int)

# Motion regime classification based on direction and speed
def classify_motion_regime(row):
    if pd.isna(row['storm_dir']) or pd.isna(row['storm_speed']):
        return np.nan
    
    speed = row['storm_speed']
    direction = row['storm_dir']
    
    if speed < 5:
        return 2
    elif 225 <= direction <= 315 or (direction <= 45) or (direction >= 315):
        return 0
    else:
        return 1

df['motion_regime'] = df.apply(classify_motion_regime, axis=1)

# Storm stage encoding from nature column
def encode_storm_stage(nature):
    if pd.isna(nature):
        return np.nan
    mapping = {'DS': 0, 'NR': 1, 'SS': 1, 'TS': 2, 'MX': 3, 'ET': 4}
    return mapping.get(nature, np.nan)

if 'nature' in df.columns:
    df['storm_stage'] = df['nature'].apply(encode_storm_stage)

# Landfall proximity features
if 'dist2land' in df.columns:
    df['is_approaching_land'] = (df['dist2land'] < 200).astype(int)
    df['land_gradient'] = df.groupby('sid')['dist2land'].diff() * -1  # Negative means approaching

# Beta-drift proxy: Coriolis-related drift (approximation)
df['beta_drift_lat'] = df['hemisphere'] * np.sin(np.radians(df['lat'].abs())) * 0.1
df['beta_drift_lon'] = df['hemisphere'] * np.sign(df['lat']) * np.cos(np.radians(df['lat'].abs())) * 0.05

# Smoothed velocity using 3-point moving average (reduces noise)
df['v_lat_smooth'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=3, center=True, min_periods=1).mean())
df['v_lon_smooth'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=3, center=True, min_periods=1).mean())

# Autoregressive motion features: recent motion averages
df['v_lat_avg_6h'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=2, min_periods=1).mean())
df['v_lon_avg_6h'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=2, min_periods=1).mean())
df['v_lat_avg_12h'] = df.groupby('sid')['v_lat'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df['v_lon_avg_12h'] = df.groupby('sid')['v_lon'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Select key columns for Kalman filter
state_cols = ['sid', 'iso_time', 'lat', 'lon', 'v_lat', 'v_lon', 'storm_speed', 'storm_dir']
feature_cols = ['track_curvature', 'latitude_regime', 'hemisphere', 'motion_regime', 
                'storm_age_hours', 'day_of_year', 'month']

if 'storm_stage' in df.columns:
    feature_cols.append('storm_stage')
if 'is_approaching_land' in df.columns:
    feature_cols.extend(['is_approaching_land', 'land_gradient'])
if 'dist2land' in df.columns:
    feature_cols.append('dist2land')

feature_cols.extend(['beta_drift_lat', 'beta_drift_lon', 'v_lat_smooth', 'v_lon_smooth',
                     'v_lat_avg_6h', 'v_lon_avg_6h', 'v_lat_avg_12h', 'v_lon_avg_12h'])

if 'usa_wind' in df.columns:
    state_cols.append('usa_wind')

df_clean = df[state_cols + feature_cols + ['basin', 'nature']].copy()
df_clean = df_clean.dropna(subset=['lat', 'lon', 'v_lat', 'v_lon'])


In [16]:
# Save processed dataset
df_clean.to_pickle("hurricane_paths_processed.pkl")
df_clean.to_csv("hurricane_paths_processed.csv", index=False)


In [17]:
# Validate data quality
print(f"Processed dataset: {len(df_clean):,} observations")
print(f"Unique storms: {df_clean['sid'].nunique():,}")
print(f"Missing values in state variables: {df_clean[['lat', 'lon', 'v_lat', 'v_lon']].isnull().sum().sum()}")
print(f"Date range: {df_clean['iso_time'].min()} to {df_clean['iso_time'].max()}")


Processed dataset: 721,960 observations
Unique storms: 13,450
Missing values in state variables: 0
Date range: 1842-10-25 03:00:00 to 2025-11-23 00:00:00
